In [48]:
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import datetime

import logging

def get_historic_data(date: datetime.datetime):
    
    date_str = date.strftime('%Y-%m-%d')
    lookup_URL = "https://www.wunderground.com/history/daily/jp/tokyo/RJTT/date/{0}".format(date_str)
    print(lookup_URL)
    options = webdriver.ChromeOptions();
    options.add_argument('headless'); # to run chrome in the backbroung
    driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options=options)  # Optional argument, if not specified will search path.
    xpath = "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr[*]"
    driver.get(lookup_URL)
    rows = WebDriverWait(driver, 1).until(EC.visibility_of_all_elements_located((By.XPATH, xpath)))
    raw_data = []
    for row in rows:
        cols=row.find_elements_by_tag_name("td")
        data_dict = {"Time":cols[0].text, 
                     "Temperature":cols[1].text, 
                     "Dew Point":cols[2].text,
                     "Humidity":cols[3].text,
                     "Wind":cols[4].text,
                     "Wind Speed":cols[5].text, 
                     "Wind Gust":cols[6].text,
                     "Pressure":cols[7].text, 
                     "Precip.":cols[8].text,
                     "Condition":cols[9].text
                    }
        raw_data.append(data_dict)
    
    df = pd.DataFrame.from_records(raw_data)
    return df

In [67]:
import io
from PIL import Image

def get_weather_elements():
    lookup_URL = "https://www.wunderground.com/weather/jp/tokyo/RJTT"
    options = webdriver.ChromeOptions();
    options.add_argument('headless'); # to run chrome in the backbroung
    driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options=options)  # Optional argument, if not specified will search path.
    xpath = "/html/body/app-root/app-today/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[3]/div[1]/div/div[1]/div[1]/lib-city-current-conditions"    
    driver.get(lookup_URL)
    html = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, xpath)))
    return html


image = get_weather_elements()[0].text
image

"access_time 7:12 PM JST on January 14, 2021 (GMT +9) | Updated just now\n--° | 39°\n53 F\nLIKE 53°\nFair\nN\n2\nTomorrow's temperature is forecast to be MUCH COOLER than today."

In [46]:
import datetime

dt = datetime.datetime(2021, 1, 1)
end = datetime.datetime.now()
step = datetime.timedelta(days = 1)

dates = []

while dt < end:
    dates.append(dt)
    dt += step

In [50]:
df = pd.DataFrame()
for date in dates[-1:]:
    print("checking date"+date.strftime("%m/%d/%Y, %H:%M:%S"))
    df_temp = get_historic_data(date)
    df_temp["Time"] = pd.to_datetime(df_temp['Time']).apply(lambda x: x.replace(year=date.year, month=date.month, day=date.day))

    df = pd.concat([df, df_temp], ignore_index=True)
df

checking date01/14/2021, 00:00:00
https://www.wunderground.com/history/daily/jp/tokyo/RJTT/date/2021-01-14


,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2021-01-01 00:00:00,37 F,14 F,38 %,VAR,2 mph,0 mph,29.83 in,0.0 in,Fair
1,2021-01-01 00:30:00,37 F,16 F,41 %,WNW,5 mph,0 mph,29.80 in,0.0 in,Fair
2,2021-01-01 01:00:00,37 F,16 F,41 %,W,5 mph,0 mph,29.83 in,0.0 in,Fair
3,2021-01-01 01:30:00,36 F,16 F,44 %,WNW,5 mph,0 mph,29.83 in,0.0 in,Fair
4,2021-01-01 02:00:00,39 F,14 F,36 %,W,5 mph,0 mph,29.83 in,0.0 in,Fair
...,...,...,...,...,...,...,...,...,...,...
660,2021-01-14 15:30:00,57 F,34 F,41 %,S,8 mph,0 mph,29.98 in,0.0 in,Fair
661,2021-01-14 16:00:00,57 F,32 F,38 %,S,8 mph,0 mph,29.98 in,0.0 in,Fair
662,2021-01-14 16:30:00,52 F,43 F,71 %,S,5 mph,0 mph,30.01 in,0.0 in,Fair
663,2021-01-14 17:00:00,50 F,46 F,87 %,ESE,5 mph,0 mph,30.01 in,0.0 in,Fair


In [51]:
pickle.dump(df, open( "test_weather_df.p", "wb" ) )

In [106]:
import pickle

df = pickle.load(open("jackson_hole_weather_df.p","rb"))

# Prepare data for uploading to mongodb

In [107]:
def convert_to_numeric(df,col):
    df[col]=df[col].str.extract(r'(\d+)')
    df[col] = pd.to_numeric(df[col])

In [108]:
def prep_for_mongo(df):
    convert_to_numeric(df,"Wind Speed")
    convert_to_numeric(df,"Dew Point")
    convert_to_numeric(df,"Humidity")
    convert_to_numeric(df,"Wind Gust")
    convert_to_numeric(df,"Pressure")
    convert_to_numeric(df,"Precip.")
    convert_to_numeric(df,"Temperature")

    df = df.rename(columns={"Precip.": "Precipitation"})
    return df

In [109]:
df = prep_for_mongo(df)

In [110]:
df.dtypes

Time             datetime64[ns]
Temperature               int64
Dew Point                 int64
Humidity                  int64
Wind                     object
Wind Speed                int64
Wind Gust                 int64
Pressure                  int64
Precipitation             int64
Condition                object
dtype: object

In [101]:
df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precipitation,Condition
0,2021-01-01 00:56:00,12,9,88,NNE,6,0,23,0,Cloudy
1,2021-01-01 01:56:00,13,10,88,NNE,5,0,23,0,Cloudy
2,2021-01-01 02:56:00,13,10,88,N,6,0,23,0,Light Snow
3,2021-01-01 03:11:00,12,10,92,N,7,0,23,0,Light Snow
4,2021-01-01 03:56:00,14,10,84,N,6,0,23,0,Light Snow


# Now upload to mongodb

In [193]:
from pymongo import MongoClient
import urllib.parse

# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)
db = client["weather"]
mycol = db["jackson_hole"]

data_dict = df.to_dict("records")

# Insert collection
mycol.insert_many(data_dict)

# Now retrieve data from mongodb

In [68]:
pipeline = [
    {
        '$addFields': {
            'datetime': {
                '$toDate': {
                    '$multiply': [
                        '$counted_at', 1000
                    ]
                }
            }
        }
    }, {
        '$addFields': {
            'counted_at': {
                '$convert': {
                    'input': '$counted_at', 
                    'to': 'double', 
                    'onError': None
                }
            }, 
            'computer_id': {
                '$convert': {
                    'input': '$computer_id', 
                    'to': 'string', 
                    'onError': None
                }
            }
        }
    }, {
        '$match': {
            'arg_file': {
                '$in': [
                    'jackson_hole.env'
                ]
            }, 
            'type': {
                '$nin': []
            }, 
            'computer_id': {
                '$in': [
                    None, '', '00000000-0000-0000-0000-3cecef225486', '00000000-0000-0000-0000-f894c218ae25'
                ]
            }, 
        }
    }, {
        '$addFields': {
            'datetime': {
                '$cond': {
                    'if': {
                        '$eq': [
                            {
                                '$type': '$datetime'
                            }, 'date'
                        ]
                    }, 
                    'then': '$datetime', 
                    'else': None
                }
            }
        }
    }, {
        '$addFields': {
            '__alias_0': {
                'year': {
                    '$year': '$datetime'
                }, 
                'month': {
                    '$subtract': [
                        {
                            '$month': '$datetime'
                        }, 1
                    ]
                }, 
                'date': {
                    '$dayOfMonth': '$datetime'
                }, 
                'hours': {
                    '$hour': '$datetime'
                }, 
                'minutes': {
                    '$minute': '$datetime'
                }
            }
        }
    }, {
        '$group': {
            '_id': {
                '__alias_0': '$__alias_0'
            }, 
            '__alias_1': {
                '$sum': {
                    '$cond': [
                        {
                            '$ne': [
                                {
                                    '$type': '$datetime'
                                }, 'missing'
                            ]
                        }, 1, 0
                    ]
                }
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            '__alias_0': '$_id.__alias_0', 
            '__alias_1': 1
        }
    }, {
        '$project': {
            'x': '$__alias_0', 
            'y': '$__alias_1', 
            '_id': 0
        }
    }, {
        '$sort': {
            'x.year': 1, 
            'x.month': 1, 
            'x.date': 1, 
            'x.hours': 1, 
            'x.minutes': 1
        }
    }
]

In [69]:
# init mongo connection
from pymongo import MongoClient
import urllib.parse
import datetime
import pandas as pd

username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)

def get_data(pipeline):
    traffic_results = client["traffic"]["traffic_data"].aggregate(pipeline)

    traffic_results = list(traffic_results)
    traffic_results.pop(0)

    for i in traffic_results:
        try:
            i["x"] = datetime.datetime(year = i["x"]["year"], month = i["x"]["month"]+1, day = i["x"]["date"],hour = i["x"]["hours"])
        except TypeError:
            pass

    traffic_df = pd.DataFrame.from_records(traffic_results)
    return traffic_df

def gen_pipeline(env_file):
    a = pipeline.copy()
    a[2]["$match"]["arg_file"]["$in"][0] = env_file
    return a

def remove_duplicates(df):
    return df.groupby(['x'], as_index=False)['y'].sum()
    #return df.sort_values('y').drop_duplicates('x', keep='last')

In [87]:
jackson_traffic_df = get_data(gen_pipeline("jackson_hole.env"))
jackson_traffic_df = remove_duplicates(jackson_traffic_df)

casa_grande_traffic_df = get_data(gen_pipeline("casa_grande.env"))
casa_grande_traffic_df = remove_duplicates(casa_grande_traffic_df)

tokyo_crosswalk_traffic_df = get_data(gen_pipeline("tokyo_crosswalk.env"))
tokyo_crosswalk_traffic_df = remove_duplicates(tokyo_crosswalk_traffic_df)


In [88]:
jackson_traffic_df

,x,y
0,2021-01-04 10:00:00,18
1,2021-01-04 11:00:00,42
2,2021-01-04 12:00:00,98
3,2021-01-04 13:00:00,157
4,2021-01-04 14:00:00,250
...,...,...
237,2021-01-14 09:00:00,9
238,2021-01-14 10:00:00,22
239,2021-01-14 11:00:00,29
240,2021-01-14 12:00:00,67


In [73]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

fig = make_subplots(rows=3, cols=1,shared_xaxes=True,)

fig.append_trace(go.Bar(
    
    x = jackson_traffic_df["x"],
    y = jackson_traffic_df["y"],
    name="Jackson Hole traffic"

), row=1, col=1)

fig.append_trace(go.Bar(
    x = casa_grande_traffic_df["x"],
    y = casa_grande_traffic_df["y"],
    name = "Casa Grande Traffic"
), row=2, col=1)

fig.append_trace(go.Bar(
    x = tokyo_crosswalk_traffic_df["x"],
    y = tokyo_crosswalk_traffic_df["y"],
    name = "Tokyo Cross Walk"

), row=3, col=1)


fig.update_layout(title_text="Human and Vehicle traffic data")
fig.show()

# Now get back the weather data from the db

In [218]:
pipeline_weather = [
  {
    "$group": {
      "_id": {
        "__alias_0": "$Time",
        "__alias_1": "$Dew Point",
        "__alias_2": "$Wind Gust",
        "__alias_3": "$Wind Speed",
        "__alias_4": "$Temperature",
        "__alias_5": "$Pressure"
      }
    }
  },
  {
    "$project": {
      "_id": 0,
      "__alias_0": "$_id.__alias_0",
      "__alias_1": "$_id.__alias_1",
      "__alias_2": "$_id.__alias_2",
      "__alias_3": "$_id.__alias_3",
      "__alias_4": "$_id.__alias_4",
      "__alias_5": "$_id.__alias_5"
    }
  },
  {
    "$project": {
      "x": "$__alias_0",
      "y": "$__alias_1",
      "y_series_0": "$__alias_2",
      "y_series_1": "$__alias_3",
      "y_series_2": "$__alias_4",
      "y_series_3": "$__alias_5",
      "_id": 0
    }
  },
  {
    "$sort": {
      "x": 1,
      "y": 1,
      "y_series_0": 1,
      "y_series_1": 1,
      "y_series_2": 1,
      "y_series_3": 1
    }
  },
  {
    "$addFields": {
      "__multi_series": {
        "$objectToArray": {
          "Wind Gust": "$y_series_0",
          "Wind Speed": "$y_series_1",
          "Temperature": "$y_series_2",
          "Pressure": "$y_series_3",
          "Dew Point": "$y"
        }
      }
    }
  },
  {
    "$unwind": "$__multi_series"
  },
  {
    "$addFields": {
      "color": "$__multi_series.k",
      "y": "$__multi_series.v"
    }
  },
  {
    "$project": {
      "__multi_series": 0,
      "y_series_0": 0,
      "y_series_1": 0,
      "y_series_2": 0,
      "y_series_3": 0
    }
  },
  {
    "$limit": 50000
  }
]

In [219]:
weather_results = client["weather"]["jackson_hole"].aggregate(pipeline_weather)
list(weather_results)

[{'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 0, 'color': 'Wind Gust'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 6, 'color': 'Wind Speed'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 12, 'color': 'Temperature'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 23, 'color': 'Pressure'},
 {'x': datetime.datetime(2021, 1, 1, 0, 56), 'y': 9, 'color': 'Dew Point'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 0, 'color': 'Wind Gust'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 5, 'color': 'Wind Speed'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 13, 'color': 'Temperature'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 23, 'color': 'Pressure'},
 {'x': datetime.datetime(2021, 1, 1, 1, 56), 'y': 10, 'color': 'Dew Point'},
 {'x': datetime.datetime(2021, 1, 1, 2, 56), 'y': 0, 'color': 'Wind Gust'},
 {'x': datetime.datetime(2021, 1, 1, 2, 56), 'y': 6, 'color': 'Wind Speed'},
 {'x': datetime.datetime(2021, 1, 1, 2, 56), 'y': 13, 'color': 'Temperature'},

In [19]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

fig = px.line(df, x="Time", y=df[["Temperature","Dew Point","Wind Speed","Wind Gust","Pressure","Precipitation"]].columns)
fig 
fig.show()

In [88]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'


values = ["Temperature","Wind Speed","Wind Gust"]
fig = make_subplots(rows=len(values)+2, cols=1)

for i in range(len(values)):

    fig.append_trace(go.Scatter(

        x = df["Time"],
        y = df[values[i]],
        mode='lines',

        name=values[i]
    ), row=i+1, col=1)

fig.append_trace(go.Scatter(
    
    x = df["Time"],
    y = df["Condition"],
    name="Condition",
    mode="markers",

), row=len(values)+1, col=1)    
    
fig.append_trace(go.Bar(
    
    x = jackson_traffic_df["x"],
    y = jackson_traffic_df["y"],
    name="Jackson Hole traffic"

), row=len(values)+2, col=1)

fig.update_layout(title_text="Human and Vehicle traffic data combined with weather data for Jackson Hole")
fig.show()

In [81]:
jackson_traffic_df.head()

,x,y
0,2021-01-04 10:00:00,18
1,2021-01-04 11:00:00,42
2,2021-01-04 12:00:00,98
3,2021-01-04 13:00:00,157
4,2021-01-04 14:00:00,250


In [89]:
jackson_traffic_df=jackson_traffic_df.set_index("x")

In [90]:
upsampled = jackson_traffic_df.resample('10Min')
interpolated = upsampled.interpolate(method='linear')

In [91]:
interpolated

,y
x,
2021-01-04 10:00:00,18.000000
2021-01-04 10:10:00,22.000000
2021-01-04 10:20:00,26.000000
2021-01-04 10:30:00,30.000000
2021-01-04 10:40:00,34.000000
...,...
2021-01-14 12:20:00,66.333333
2021-01-14 12:30:00,66.000000
2021-01-14 12:40:00,65.666667


In [165]:
df=df.set_index("Time")
df

KeyError: "None of ['Time'] are in the columns"

In [175]:
df["Temperature"]

Time
2021-01-01 00:56:00    12
2021-01-01 01:56:00    13
2021-01-01 02:56:00    13
2021-01-01 03:11:00    12
2021-01-01 03:56:00    14
                       ..
2021-01-13 17:56:00    31
2021-01-13 18:05:00    30
2021-01-13 18:15:00    30
2021-01-13 18:56:00    29
2021-01-13 19:56:00    28
Name: Temperature, Length: 609, dtype: int64

In [171]:
upsampled_df = df["Temperature"].resample('10Min')

In [172]:
interpolated_df = upsampled_df.interpolate(method='linear')

In [174]:
interpolated_df

Time
2021-01-01 00:50:00     NaN
2021-01-01 01:00:00     NaN
2021-01-01 01:10:00     NaN
2021-01-01 01:20:00     NaN
2021-01-01 01:30:00     NaN
                       ... 
2021-01-13 19:10:00    36.0
2021-01-13 19:20:00    36.0
2021-01-13 19:30:00    36.0
2021-01-13 19:40:00    36.0
2021-01-13 19:50:00    36.0
Freq: 10T, Name: Temperature, Length: 1843, dtype: float64

In [140]:
interpolated_df

Time
2021-01-01 00:50:00     NaN
2021-01-01 01:00:00     NaN
2021-01-01 01:10:00     NaN
2021-01-01 01:20:00     NaN
2021-01-01 01:30:00     NaN
                       ... 
2021-01-13 19:10:00    36.0
2021-01-13 19:20:00    36.0
2021-01-13 19:30:00    36.0
2021-01-13 19:40:00    36.0
2021-01-13 19:50:00    36.0
Freq: 10T, Name: Temperature, Length: 1843, dtype: float64

In [154]:
vehicle_inter = interpolated.loc['2021-1-6':'2021-1-12']["y"]
vehicle_inter

x
2021-01-06 00:00:00    374.000000
2021-01-06 00:10:00    380.166667
2021-01-06 00:20:00    386.333333
2021-01-06 00:30:00    392.500000
2021-01-06 00:40:00    398.666667
                          ...    
2021-01-12 23:10:00    371.000000
2021-01-12 23:20:00    407.000000
2021-01-12 23:30:00    443.000000
2021-01-12 23:40:00    479.000000
2021-01-12 23:50:00    515.000000
Freq: 10T, Name: y, Length: 1008, dtype: float64

In [151]:
temperature_inter = interpolated_df.loc['2021-1-6':'2021-1-12']
temperature_inter

Time
2021-01-06 00:00:00    15.801471
2021-01-06 00:10:00    15.647059
2021-01-06 00:20:00    15.492647
2021-01-06 00:30:00    15.338235
2021-01-06 00:40:00    15.183824
                         ...    
2021-01-12 23:10:00    26.440678
2021-01-12 23:20:00    26.525424
2021-01-12 23:30:00    26.610169
2021-01-12 23:40:00    26.694915
2021-01-12 23:50:00    26.779661
Freq: 10T, Name: Temperature, Length: 1008, dtype: float64

In [156]:
temperature_inter.corr(vehicle_inter, "pearson")

0.030053754896635473

In [176]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])




fig.add_trace(
    go.Scatter(x=temperature_inter.index, y= temperature_inter,
                        mode='lines',
                        name='Temperature'),
    secondary_y=False,

)

fig.add_trace(
    go.Scatter(x=vehicle_inter.index, y= vehicle_inter,
                        mode='lines',
                        name='Traffic'),
        secondary_y=True,

)


# Set y-axes titles
fig.update_yaxes(title_text="Temperature", secondary_y=False)
fig.update_yaxes(title_text="Traffic Volume", secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Correlation between Temperature and vehicular traffic in Jackson Hole"
)

fig.show()

In [221]:
fig = px.line(x = traffic_df["x"], y = traffic_df["y"])
fig 
fig.show()

In [74]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Temperature"],
                        mode='lines',
                        name='Temperature'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Wind Gust"],
                        mode='lines',
                        name='Wind Gust'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Wind Gust"],
                        mode='lines',
                        name='Wind Gust'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x = traffic_df["x"], y = traffic_df["y"],
                        mode='lines',
                        name='Traffic volume'),    

    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [207]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

fig = go.Figure()


# Add traces
fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Temperature"],
                        mode='lines',
                        name='Temperature',
               yaxis="y2"
              )
)


fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Condition"],
                        mode='lines',
                        name='Condition',
                        yaxis="y3")
)

fig.add_trace(
    go.Bar(x = traffic_df["x"], y = traffic_df["y"],
                        name='Traffic volume'),
)


# Create axis objects
fig.update_layout(
    yaxis=dict(
        title="yaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="yaxis2 title",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.15
    ),
    yaxis3=dict(
        title="yaxis3 title",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="yaxis4 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85
    )
)

# Update layout properties
fig.update_layout(
    title_text="multiple y-axes example",
)
fig.show()

In [ ]:
# /html/body/app-root/app-today/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[3]/div[1]/div/div[1]/div[1]/lib-city-current-conditions/div